In [ ]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install osgeo

In [ ]:
import os
from osgeo import gdal, osr

# --- Reference Raster Setup ---
reference_tif = "/content/drive/MyDrive/GHS_POP_DATA/GHS_POP_E1975_GLOBE_R2023A_4326_30ss_V1_0.tif"

ref_ds = gdal.Open(reference_tif)
if ref_ds is None:
    raise ValueError(f"Could not open reference TIFF file: {reference_tif}")

ref_proj = ref_ds.GetProjection()
ref_geotrans = ref_ds.GetGeoTransform()
ref_srs = osr.SpatialReference()
ref_srs.ImportFromWkt(ref_proj)

ref_x_min, ref_pixel_width, _, ref_y_max, _, ref_pixel_height = ref_ds.GetGeoTransform()
ref_x_max = ref_x_min + (ref_ds.RasterXSize * ref_pixel_width)
ref_y_min = ref_y_max + (ref_ds.RasterYSize * ref_pixel_height)

# --- Input/Output Directories ---
input_dir = '/content/drive/MyDrive/hwi_stats/dgca_back'
output_dir = '/content/drive/MyDrive/CCRI/scratch'

# --- Reprojection Loop ---
for year in range(1960, 2030, 10):
    tif_path = os.path.join(input_dir, f'average_hwi_{year}s.tif')
    reprojected_path = os.path.join(output_dir, f'average_hwi_{year}s.tif')

    # Check if input file exists and is a valid GDAL dataset
    input_ds = gdal.Open(tif_path)
    if input_ds is None:
        print(f"Error opening input file: {tif_path}. Skipping.")
        continue

    # Check if output directory exists, create it if not
    os.makedirs(output_dir, exist_ok=True)

    try:
        # Reproject only if the file is valid
        gdal.Warp(
            reprojected_path,
            input_ds,
            dstSRS=ref_srs,
            outputBounds=[ref_x_min, ref_y_min, ref_x_max, ref_y_max],
            xRes=ref_pixel_width,
            yRes=ref_pixel_height
        )
    except Exception as e:
        print(f"Error reprojecting {tif_path}: {e}")
    finally:
        input_ds = None  # Close the input file

print("Reprojection completed.")